In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import fetch_lfw_people
import numpy as np
import cv2.cv2 as cv2

def data():
    lfw_people = fetch_lfw_people(
        data_home="drive/My Drive/Colab Notebooks/Resnet_Face_Detection/scikit_learn_data",
        min_faces_per_person=14,
        color=True,
        slice_=(slice(0, 250, None), slice(0, 250, None)),
    )

    # The original images are 250 x 250 pixels,
    # but the default slice and resize arguments reduce them to 62 x 47 pixels.

    # introspect the images arrays to find the shapes
    n_samples, h, w, pixel = lfw_people.images.shape

    X = lfw_people.images

    # the label to predict is the id of the person
    y = lfw_people.target
    target_names = lfw_people.target_names
    n_classes = target_names.shape[0]

    print("Total dataset size:")
    print("n_samples: %d" % n_samples)
    # print("n_features: %d" % n_features)
    print("n_classes: %d" % n_classes)
    print("Image size: %dx%d" % (h, w))

    # split into a training and testing set
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, shuffle=True 
    )

    return X_train, X_test, y_train, y_test, n_classes


X_train, X_test, y_train, y_test, n_classes = data()


Total dataset size:
n_samples: 3735
n_classes: 106
Image size: 125x125


In [12]:
from keras.applications.resnet50 import ResNet50#, preprocess_input
# from keras import optimizers
from keras.models import Sequential#, Model, load_model
from keras.layers import (
    Dense,
)
    # Dropout,
    # Flatten,
    # GlobalAveragePooling2D,
    # BatchNormalization,
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

img_height, img_width = X_train[0].shape[0], X_train[0].shape[1]
num_classes = n_classes
epochs=20
batch_size=32

model = Sequential()

model.add(
    ResNet50(
        include_top=False,
        weights="imagenet",
        pooling="avg",
        input_shape=(img_height, img_width, 3)
    )
)
model.add(Dense(num_classes, activation="softmax", name="softmax1"))

earlyStopping = EarlyStopping(
    monitor="val_loss", restore_best_weights=True, patience=4, verbose=0, mode="min"
)
# log = CSVLogger("drive/My Drive/Colab Notebooks/Resnet_Face_Detection/resnet_logs.csv")
lr_reduce = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

datagen = ImageDataGenerator(
    horizontal_flip=True,
   	fill_mode="nearest",
    shear_range=0.1,
    zoom_range=0.1
)

# model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_data=(X_test, y_test), callbacks=[earlyStopping, log, lr_reduce])
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), validation_data=(X_test, y_test),callbacks=[earlyStopping, lr_reduce],
                    steps_per_epoch=len(X_train) / batch_size, epochs=epochs)


model.save("drive/My Drive/Colab Notebooks/Resnet_Face_Detection/model.h5", overwrite=True)

Epoch 1/20
100/99 [==============================] - 77s 774ms/step - loss: 3.2220 - accuracy: 0.3166 - val_loss: 5.0284 - val_accuracy: 0.0802
Epoch 2/20
100/99 [==============================] - 58s 580ms/step - loss: 1.5064 - accuracy: 0.6084 - val_loss: 4.2232 - val_accuracy: 0.1159
Epoch 3/20
100/99 [==============================] - 58s 579ms/step - loss: 0.8534 - accuracy: 0.7637 - val_loss: 3.2656 - val_accuracy: 0.3387
Epoch 4/20
100/99 [==============================] - 58s 581ms/step - loss: 0.5310 - accuracy: 0.8459 - val_loss: 2.5734 - val_accuracy: 0.4189
Epoch 5/20
100/99 [==============================] - 58s 580ms/step - loss: 0.3991 - accuracy: 0.8891 - val_loss: 2.0622 - val_accuracy: 0.5152
Epoch 6/20
100/99 [==============================] - 58s 580ms/step - loss: 0.2895 - accuracy: 0.9178 - val_loss: 2.9749 - val_accuracy: 0.3672
Epoch 7/20
100/99 [==============================] - 58s 580ms/step - loss: 0.1848 - accuracy: 0.9502 - val_loss: 1.5488 - val_accuracy:

In [13]:
preds = model.evaluate(X_test, y_test)
print("\nLoss = " + str(preds[0]))
print("Test Accuracy = " + str(100*preds[1]))
model.summary()

561/561 [==============================] - 3s 5ms/step

Loss = 0.4579507594865061
Test Accuracy = 88.41354846954346
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
softmax1 (Dense)             (None, 106)               217194    
Total params: 23,804,906
Trainable params: 23,751,786
Non-trainable params: 53,120
_________________________________________________________________
